In [1]:
!pip install seaborn

     |████████████████████████████████| 292 kB 27.6 MB/s            


In [2]:
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import *
from sklearn.feature_extraction.text import *

import konlpy

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer, TFBertForSequenceClassification

In [4]:
def preprocess_spam(path):
    import pandas as pd
    from konlpy.tag import Mecab
    df = pd.read_csv(path)
    
    # 결측치 제거
    df.dropna(how='any', inplace=True)
    
    # text 데이터 변경
    df.text = df.text.replace(r'[^\]가-힣]', r'', regex=True)
    df.text = df.text.apply(Mecab().nouns)
    df.text = df.text.apply(lambda x: ' '.join(x))
    
    return df

In [5]:
df = preprocess_spam('data/spam.csv')

In [6]:
df

,text,label
0,발신 박춘규 회원 손절 주식 월급 배 법,spam
1,발신 골든 브릿지 손실,spam
2,발신 대박 원입 코드,spam
3,발신 광고 이정미 주식,spam
4,발신 송승용 사람 곳 이유 하루 정보 공개,spam
...,...,...
20095,발신 상투 자황 기봉 이재훈 투자 컨 설 담당자 배정 완료 담당자 확인,spam
20096,발신 광고 조 마충 주소 카톡 무료 거부,spam
20097,발신 김 환택 지금 긴급 정보,spam
20098,발신 년 노하우 명품 구매 대행 스토리 사랑 성원 감사,ham


In [7]:
df.columns.tolist()

['text', 'label']

In [8]:
le = LabelEncoder()
le.fit(df.columns)
df.label = le.fit_transform(df.label)
df

,text,label
0,발신 박춘규 회원 손절 주식 월급 배 법,1
1,발신 골든 브릿지 손실,1
2,발신 대박 원입 코드,1
3,발신 광고 이정미 주식,1
4,발신 송승용 사람 곳 이유 하루 정보 공개,1
...,...,...
20095,발신 상투 자황 기봉 이재훈 투자 컨 설 담당자 배정 완료 담당자 확인,1
20096,발신 광고 조 마충 주소 카톡 무료 거부,1
20097,발신 김 환택 지금 긴급 정보,1
20098,발신 년 노하우 명품 구매 대행 스토리 사랑 성원 감사,0


In [42]:
df

,text,label
0,발신 박춘규 회원 손절 주식 월급 배 법,1
1,발신 골든 브릿지 손실,1
2,발신 대박 원입 코드,1
3,발신 광고 이정미 주식,1
4,발신 송승용 사람 곳 이유 하루 정보 공개,1
...,...,...
20095,발신 상투 자황 기봉 이재훈 투자 컨 설 담당자 배정 완료 담당자 확인,1
20096,발신 광고 조 마충 주소 카톡 무료 거부,1
20097,발신 김 환택 지금 긴급 정보,1
20098,발신 년 노하우 명품 구매 대행 스토리 사랑 성원 감사,0


In [10]:
# model = AutoModel.from_pretrained("klue/bert-base")
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [11]:
train_encodings = tokenizer(df.text.tolist(), truncation=True, padding=True, max_length=52)

In [45]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    df.label
))

In [13]:
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2, from_pt=True)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.compile(optimizer='adam', loss=model.compute_loss, metrics=['accuracy'])

In [25]:
es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                   min_delta=0.01,
                                   patience=2,
                                   restore_best_weights=True,
                                  )

In [48]:
model.fit(train_dataset.shuffle(1000).batch(16),
          epochs=10, batch_size=16,
          )

Epoch 1/10


TypeError: 'NoneType' object is not callable

In [24]:
!nvidia-smi

No devices were found


In [17]:
from keras import backend as K

In [18]:
K.tensorflow_backend._get_available_gpus()

AttributeError: module 'tensorflow._api.v2.config' has no attribute 'experimental_list_devices'

In [8]:
sns.countplot(x = spam.label)
plt.show()

NameError: name 'spam' is not defined

In [111]:
spam.loc[spam.text.isna()]

,text,label
1661,NaN,ham
1980,NaN,ham
3785,NaN,ham
4518,NaN,ham
5408,NaN,ham
11245,NaN,ham
13528,NaN,ham
14163,NaN,ham
15654,NaN,ham
18544,NaN,ham


In [112]:
spam.dropna(inplace=True)

In [113]:
spam.text = spam.text.replace(r'[^가-힣\w]+', r'', regex=True)

In [114]:
mecab = konlpy.tag.Mecab()

In [115]:
spam.text = spam.text.apply(mecab.nouns)
spam.text = spam.text.apply(lambda x:' '.join(x))

In [116]:
spam

,text,label
0,발신 박춘규 회원 손절 주식 월급 배 법,spam
1,발신 골든 브릿지 손실,spam
2,발신 대박 원입 드,spam
3,발신 광고 이정미 주식,spam
4,발신 송승용 사람 곳 이유 하루 정보 공개,spam
...,...,...
20095,발신 상투 자황 기봉 이재훈 투자 컨 설 담당자 배정 완료 담당자 확인,spam
20096,발신 광고 조마 충 矛 주소 카 톡 무료 거부,spam
20097,발신 김 환택 지금 긴급 정보,spam
20098,발 신년 노하우 명품 구매 대행 스토리 사랑 성원 감사,ham


In [117]:
spam.isna().sum()

text     0
label    0
dtype: int64

In [118]:
spam

,text,label
0,발신 박춘규 회원 손절 주식 월급 배 법,spam
1,발신 골든 브릿지 손실,spam
2,발신 대박 원입 드,spam
3,발신 광고 이정미 주식,spam
4,발신 송승용 사람 곳 이유 하루 정보 공개,spam
...,...,...
20095,발신 상투 자황 기봉 이재훈 투자 컨 설 담당자 배정 완료 담당자 확인,spam
20096,발신 광고 조마 충 矛 주소 카 톡 무료 거부,spam
20097,발신 김 환택 지금 긴급 정보,spam
20098,발 신년 노하우 명품 구매 대행 스토리 사랑 성원 감사,ham


In [50]:
spam.text

0                 Web발신 박춘규회원님손절주식은그만월급배만드는법http lco jp eA
1             Web발신 골든브릿지에서손실을보셨나요 http bitly kr bRGtq FW 
2                             Web발신 대 박 가 원 입코 WN 드MEP com
3         Web발신 광고 이정미님아직도주식하시나요http pf kakao com _uHxlgj 
4              Web발신 송승용님사람이모이는곳에는이유가있습니다하루 정보공개is gd JsJP
                               ...                        
20095     Web발신 이상투자황기봉님 이재훈 투자컨설담당자배정완료 담당자확인 http bit...
20096                 Web발신 광고 조마 峙 충 矛 까지 주소Joma 카JM톡무료거부
20097     Web발신 김환택님지금이라도늦지않았습니다 월긴급정보http pf kakao com...
20098     Web발신 년노하우명품구매대행논스토리nonstory com항상아낌없는사랑과성원에감...
20099            Web발신 최현님월일특급정보 https pf kakao com _SfcsT
Name: text, Length: 20089, dtype: object

In [15]:
len(le.classes_)

2

In [50]:
%cp -r '/aihub/workspace/data' '/aihub/data/'